## Plot LFR benchmark results

In this notebook we plot the benchmark results, comparing the community detection methods under
consideration.

First, we import the necessary packages and initialize the global plotting parameters.

In [ ]:
%reload_ext autoreload
%autoreload 2

from src.scripts.plot_lfr import plot_ami_vs_mu, plot_ami_vs_rho,plot_synwalk_error_vs_mu, init_plot_style
%pylab

init_plot_style()

### Plot AMI results
Here we plot the benchmark AMI results for a fixed average degree and network size $n$ as a function of the mixing
parameter $\mu$.

In [ ]:
# plot setup
avg_degrees = [15, 25, 50] # average node degrees
network_sizes = [300, 600, 1200] # network sizes
save_figures = True # if True, we save the figures as .pdf

plot_legend = True
for avg_degree in avg_degrees:
    for n in network_sizes:
        plot_ami_vs_mu(avg_degree, n, save_figure=save_figures, plot_legend=plot_legend)
        plot_legend = False

Here we plot the benchmark AMI results for a fixed average degree and mixing parameter $\mu$ as a function of the network density $\rho$.

In [ ]:
# plot setup
avg_degrees = [15, 25, 50] # average node degrees
mixing_parameters = [0.35, 0.45, 0.55] # network sizes
save_figures = True # if True, we save the figures as .pdf

plot_legend = True
for avg_degree in avg_degrees:
    for mu in mixing_parameters:
        plot_ami_vs_rho(avg_degree, mu, save_figure=save_figures, plot_legend=plot_legend)
        plot_legend = False

### Plot synwalk error results

Here we plot the relative deviation of the synwalk objective for a predicted clustering
and the ground truth clustering of a graph.

In [ ]:
# plot setup
avg_degrees = [15, 25, 50] # average node degrees
network_sizes = [300, 600, 1200] # network sizes
save_figures = True # if True, we save the figures as .pdf

plot_legend = True
for avg_degree in avg_degrees:
    for n in network_sizes:
        plot_synwalk_error_vs_mu(avg_degree, n, save_figure=save_figures, plot_legend=plot_legend)
        plot_legend = False